## Developing a TCNModel with darts package

In [11]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import ta

import torch.optim as optim
#from torch.utils.tensorboard import SummaryWriter

from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, Theta
from darts.metrics import mape, rmse, r2_score
from darts.utils.statistics import check_seasonality, plot_acf
from darts.utils.timeseries_generation import datetime_attribute_timeseries

from darts.datasets import SunspotsDataset, AirPassengersDataset

In [12]:
# Import data
df= pd.read_csv("../raw_data/20230609-165037-processed-data.csv")
df

,timestamp,open,high,low,close,volume,volume_CMF,volume_FII,volume_MFI,volume_VWA,...,vortex_pos,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
0,2019-09-11 19:00:00,9992.18,10110.21,9934.11,10044.87,643.320,-0.041563,154.369057,34.613137,10037.462370,...,0.900406,3000.93,2.25,1.70,-1,-0.33674,0.02920,-1,58.337947,-1
1,2019-09-11 20:00:00,10044.14,10077.34,10034.91,10054.43,645.218,-0.077929,1013.499774,38.128005,10036.146735,...,0.971911,3000.93,2.25,1.70,-1,-0.37469,-0.33674,-1,58.113691,-1
2,2019-09-11 21:00:00,10053.77,10123.29,10053.32,10068.20,608.631,-0.116032,2065.978217,46.729583,10039.019781,...,0.998304,3000.93,2.25,1.70,1,-0.08683,-0.37469,-1,58.966328,-1
3,2019-09-11 22:00:00,10069.21,10103.43,10043.71,10077.60,726.360,-0.117717,2746.236186,44.287748,10044.120556,...,1.058558,3000.93,2.25,1.70,1,0.16529,-0.08683,-1,57.693104,-1
4,2019-09-11 23:00:00,10073.64,10125.56,10064.11,10102.87,696.837,-0.070102,4869.498301,52.811412,10053.708535,...,1.169051,3000.93,2.25,1.70,1,0.58994,0.16529,1,62.394941,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32665,2023-06-03 20:00:00,27239.40,27246.40,27101.00,27147.60,15130.376,-0.048901,45306.477416,59.148253,27189.586323,...,0.983015,4282.37,5.25,5.08,1,0.55827,0.61369,-1,53.765071,1
32666,2023-06-03 21:00:00,27147.70,27173.70,27113.90,27138.80,6229.878,-0.075151,31002.276871,58.559862,27189.451257,...,1.015887,4282.37,5.25,5.08,-1,0.15512,0.55827,-1,54.075485,1
32667,2023-06-03 22:00:00,27138.90,27153.60,27000.00,27084.90,15389.644,-0.042819,-91926.878625,49.242500,27175.830184,...,0.892200,4282.37,5.25,5.08,-1,-0.36126,0.15512,-1,44.950464,1
32668,2023-06-03 23:00:00,27084.90,27119.50,27042.00,27052.70,7760.747,-0.045307,-114493.903593,43.575625,27168.759575,...,0.905507,4282.37,5.25,5.08,-1,-0.86198,-0.36126,-1,45.530993,1


In [13]:
df= df.dropna().reset_index(drop=True)
df

,timestamp,open,high,low,close,volume,volume_CMF,volume_FII,volume_MFI,volume_VWA,...,vortex_pos,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
0,2019-09-11 19:00:00,9992.18,10110.21,9934.11,10044.87,643.320,-0.041563,154.369057,34.613137,10037.462370,...,0.900406,3000.93,2.25,1.70,-1,-0.33674,0.02920,-1,58.337947,-1
1,2019-09-11 20:00:00,10044.14,10077.34,10034.91,10054.43,645.218,-0.077929,1013.499774,38.128005,10036.146735,...,0.971911,3000.93,2.25,1.70,-1,-0.37469,-0.33674,-1,58.113691,-1
2,2019-09-11 21:00:00,10053.77,10123.29,10053.32,10068.20,608.631,-0.116032,2065.978217,46.729583,10039.019781,...,0.998304,3000.93,2.25,1.70,1,-0.08683,-0.37469,-1,58.966328,-1
3,2019-09-11 22:00:00,10069.21,10103.43,10043.71,10077.60,726.360,-0.117717,2746.236186,44.287748,10044.120556,...,1.058558,3000.93,2.25,1.70,1,0.16529,-0.08683,-1,57.693104,-1
4,2019-09-11 23:00:00,10073.64,10125.56,10064.11,10102.87,696.837,-0.070102,4869.498301,52.811412,10053.708535,...,1.169051,3000.93,2.25,1.70,1,0.58994,0.16529,1,62.394941,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32665,2023-06-03 20:00:00,27239.40,27246.40,27101.00,27147.60,15130.376,-0.048901,45306.477416,59.148253,27189.586323,...,0.983015,4282.37,5.25,5.08,1,0.55827,0.61369,-1,53.765071,1
32666,2023-06-03 21:00:00,27147.70,27173.70,27113.90,27138.80,6229.878,-0.075151,31002.276871,58.559862,27189.451257,...,1.015887,4282.37,5.25,5.08,-1,0.15512,0.55827,-1,54.075485,1
32667,2023-06-03 22:00:00,27138.90,27153.60,27000.00,27084.90,15389.644,-0.042819,-91926.878625,49.242500,27175.830184,...,0.892200,4282.37,5.25,5.08,-1,-0.36126,0.15512,-1,44.950464,1
32668,2023-06-03 23:00:00,27084.90,27119.50,27042.00,27052.70,7760.747,-0.045307,-114493.903593,43.575625,27168.759575,...,0.905507,4282.37,5.25,5.08,-1,-0.86198,-0.36126,-1,45.530993,1


In [14]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32670 entries, 0 to 32669
Data columns (total 61 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   timestamp                   32670 non-null  datetime64[ns]
 1   open                        32670 non-null  float64       
 2   high                        32670 non-null  float64       
 3   low                         32670 non-null  float64       
 4   close                       32670 non-null  float64       
 5   volume                      32670 non-null  float64       
 6   volume_CMF                  32670 non-null  float64       
 7   volume_FII                  32670 non-null  float64       
 8   volume_MFI                  32670 non-null  float64       
 9   volume_VWA                  32670 non-null  float64       
 10  mom_AOI                     32670 non-null  float64       
 11  mom_PPO                     32670 non-null  float64   

In [16]:
df['close']

0        10044.87
1        10054.43
2        10068.20
3        10077.60
4        10102.87
           ...   
32665    27147.60
32666    27138.80
32667    27084.90
32668    27052.70
32669    26990.00
Name: close, Length: 32670, dtype: float64

In [17]:
freq = '1h'
target_col = 'close'
time_col = 'timestamp'
series = TimeSeries.from_dataframe(df, time_col=time_col, value_cols=[target_col], freq=freq)

In [18]:
type(series[0])

darts.timeseries.TimeSeries

In [19]:
train, val = series.split_after(pd.Timestamp('2023-01-01 00:00:00'))

In [20]:
train

<TimeSeries (DataArray) (timestamp: 28974, component: 1, sample: 1)>
array([[[10044.87]],

       [[10054.43]],

       [[10068.2 ]],

       ...,

       [[16564.3 ]],

       [[16563.5 ]],

       [[16544.  ]]])
Coordinates:
  * timestamp  (timestamp) datetime64[ns] 2019-09-11T19:00:00 ... 2023-01-01
  * component  (component) object 'close'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [21]:
trf = Scaler()

In [22]:
# fit the transformer to the training dataset
train_trf = trf.fit_transform(train)

In [24]:
# apply the transformer to the validation set and the complete series 
val_trf = trf.transform(val)
ts_trf = trf.transform(train)

In [26]:
model = RNNModel(input_chunk_length=1, model='LSTM', output_chunk_length=2, n_epochs=200)

ignoring user defined `output_chunk_length`. RNNModel uses a fixed `output_chunk_length=1`.


In [ ]:
model.fit()

In [ ]:
# scaler = RobustScaler()
# scaler.fit(series)
# # Apply the scaling transformation to the time series
# scaled_series = series.univariate_values().apply(lambda x: scaler.transform(x.reshape(1, -1)).flatten())

In [ ]:
scaled_time_series = TimeSeries.from_times_and_values(series.time_index(), scaled_series)
scaled_time_series

In [ ]:
model = RNNModel(input_chunk_length=32, model='LSTM', output_chunk_length=2, n_epochs=100)

In [ ]:
# Fit the model to the training data
model.fit(train_scaled, verbose=True)

In [ ]:
preds = model.predict(len(val_scaled))